In [3]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')
target()

running inner()


In [5]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__=='__main__':
    main()

running register(<function f1 at 0x005F4810>)
running register(<function f2 at 0x005F4858>)
running main()
registry -> [<function f1 at 0x005F4810>, <function f2 at 0x005F4858>]
running f1()
running f2()
running f3()


In [10]:
b = 6
def f2(a):
    global b
    print(a)
    print(b)
    b = 9
f2(3)
b

3
6


9

In [13]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

avg = make_averager()

11.0

In [17]:
avg(13)

31.8

In [20]:
avg.__code__.co_varnames
avg.__code__.co_freevars

('series',)

In [25]:
avg.__closure__
avg.__closure__[0].cell_contents

[11, 10, 12, 113, 13]

In [26]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager


In [27]:
avg = make_averager()
avg(5)
avg(10)
avg(15)

10.0

In [19]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' %(elapsed, name, arg_str, result))
        return result
    return clocked

In [20]:
import time

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! = ', factorial(6))

**************************************** Calling snooze(.123)
[0.12358519s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000079s] factorial(1) -> 1
[0.00003516s] factorial(2) -> 2
[0.00005017s] factorial(3) -> 6
[0.00006440s] factorial(4) -> 24
[0.00009877s] factorial(5) -> 120
[0.00011852s] factorial(6) -> 720
6! =  720


In [24]:
factorial.__name__

'clocked'

In [25]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' %(k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pair))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' %(elapsed, name, arg_str, result))
        return result
    return clocked

In [26]:
@clock
def snooze_1(seconds):
    time.sleep(seconds)
snooze_1.__name__

'snooze_1'

In [29]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__ == '__main__':
    print(fibonacci(6))

[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00097156s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00097156s] fibonacci(4) -> 3
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(4) -> 3
[0.00099444s] fibonacci(5) -> 5
[0.00196600s] fibonacci(6) -> 8
8


In [30]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__':
    print(fibonacci(6))

[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00099874s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00099874s] fibonacci(4) -> 3
[0.00000000s] fibonacci(5) -> 5
[0.00099874s] fibonacci(6) -> 8
8


In [1]:
from functools import singledispatch
from collections import abc
import numbers
import html

#写成分派函数
#python 不直接支持重载， 用这种方式来达到相同效果


@singledispatch    #基函数
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):       #专门函数的名称无关紧要
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [6]:
htmlize((5,6))
htmlize(5)
htmlize('aaa')
htmlize({'a':5})#没有专门函数处理字典类型，用基函数处理

'<pre>{&#x27;a&#x27;: 5}</pre>'

参数化装饰器----装饰器工厂函数

In [12]:
registry = []

def register(func):
    print('running register(%s)' %func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
print('running main()')
print('registry ->', registry)
f1()
    

running register(<function f1 at 0x05B1F930>)
running main()
registry -> [<function f1 at 0x05B1F930>]
running f1()


In [15]:
#register 函数是装饰器工厂函数，调用后返回装饰器
registry = set() #1
def register(active=True): #2
    def decorate(func): #3  真正的装饰器
        print('running register(active=%s) -> decorate(%s)'%(active, func))
        if active: #4
            registry.add(func)
        else:
            registry.discard(func) #5
        return func #6 装饰器返回函数
    return decorate #7 装饰器工厂函数返回装饰器decorate

@register(active=False) #8 装饰器工厂函数作为函数调用，记作@register()，返回装饰器@decorate
def f1():               # 若是装饰器则记作@register，无括号
    print('running f1()')

@register() #9
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
registry

running register(active=False) -> decorate(<function f1 at 0x00AE8F60>)
running register(active=True) -> decorate(<function f2 at 0x00AE8FA8>)


{<function __main__.f2()>}

In [16]:
register()(f3)

running register(active=True) -> decorate(<function f3 at 0x00AE8B70>)


<function __main__.f3()>

In [17]:
registry

{<function __main__.f2()>, <function __main__.f3()>}

In [18]:
register(active=False)(f2)

running register(active=False) -> decorate(<function f2 at 0x00AE8FA8>)


<function __main__.f2()>

In [19]:
registry

{<function __main__.f3()>}

In [26]:
#clockdeco_param.py
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT): #1
    def decorate(func):     #2
        def clocked(*_args):#3包装被装饰的函数
            t0 = time.time()
            _result = func(*_args) #4
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args) #5
            result = repr(_result) #6
            print(fmt.format(**locals()))#7 在fmt中引用clocked的局部变量
            return _result #8 返回被装饰的函数的返回值
        return clocked #9 装饰器返回装饰后的函数
    return decorate #10 返回装饰器

if __name__ == '__main__':
    @clock()
    def snooze(seconds):
        time.sleep(seconds)
    
    for i in range(3):
        snooze(.123)
            

[0.12324595s] snooze(0.123) -> None
[0.12308788s] snooze(0.123) -> None
[0.12332892s] snooze(0.123) -> None


In [27]:
import time
@clock('{name}:{elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze:0.1238102912902832s
snooze:0.12360286712646484s
snooze:0.12391042709350586s


In [28]:
import time
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.124s
